In [1]:
from sys import displayhook
from binance.client import Client
from binance.enums import *
import pandas as pd
import time
import datetime
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl

In [2]:
api_key = 'ihvXWbHvvDrYuroIJOS9ANW6X7tv5OQSYibLLloKyIGui0w93Q5yUJOBZclnNL5D'
api_secret = 'uzmKBJ3lmVS4TsTl2AOHVJOYTFfLM5qrg00vAqUyTYbSjiy7BDPlTpaq2Oy7qdo2'
client = Client(api_key, api_secret)
db_data = 'mysql+mysqldb://' + 'admin' + ':' + '0323A8E3DB' + '@' + 'coin-database-1-instance-1.c9hqcydvu19f.us-east-1.rds.amazonaws.com' + ':3306/' + 'crypto_db' + '?charset=utf8mb4'
engine = create_engine(db_data)

check_date = pd.read_sql_query("SELECT MAX(date_request) FROM crypto_trend", engine) 
date_recent_checked = str(check_date.iloc[0, 0])

trend_table_status = pd.read_sql_query("show table status like 'crypto_trend'", engine)
trend_table_rowcount_old = trend_table_status['Rows'].values[0]

cryptoString = 'SELECT * FROM crypto'
crypto_table = pd.read_sql_query(cryptoString, engine)
full_id_list = crypto_table.id.astype(int).tolist()

In [3]:
start_string_close = "2022-04-03 12:00:00" ##used 11
start_string = "2022-04-03 14:00:00" # 2 hours more than close
end_string = "2022-04-14 21:00:00"
start_date = datetime.datetime.strptime(start_string, '%Y-%m-%d %H:%M:%S')
end_date = datetime.datetime.strptime(end_string, '%Y-%m-%d %H:%M:%S')

new_close_string = f"SELECT crypto_id, date, close FROM crypto_price WHERE date >= '{start_string_close}' AND date <= '{end_string}' ORDER BY date"
closes_all = pd.read_sql_query(new_close_string, engine)

new_trends_string = f"SELECT * FROM crypto_trend WHERE date_request > '{start_string}' AND date_request < '{end_string}' ORDER BY date_request, crypto_id, date"
new_trends_all = pd.read_sql_query(new_trends_string, engine)

In [21]:
maxholds = 3
buy_threshold = 60
sell_threshold = 10

trades = [[],[],[],[]]
tradedata = []
buyPrice  = [0] * (len(full_id_list))
sellPrice = [0] * (len(full_id_list))
tradeBuy  = [0] * (len(full_id_list))
tradeSell = [0] * (len(full_id_list))
buyTime   = [0] * (len(full_id_list))
sellTime  = [0] * (len(full_id_list))
holding = [False] * (len(full_id_list))
buy_ids = []


In [22]:
tradedata = []
for i in range(int((end_date - start_date).total_seconds()/3600)):
    date_lower = str(start_date + i*datetime.timedelta(hours=1))
    date_upper = str(start_date + (i+1)*datetime.timedelta(hours=1))
    date_price_lower = str(start_date + i*datetime.timedelta(hours=1) - datetime.timedelta(hours=5))
    ############################################ TESTING ONLY ##################################################################
    
    ##new_trends_string = f"SELECT * FROM crypto_trend WHERE date_request > '{date_lower}' AND date_request < '{date_upper}' ORDER BY date_request, crypto_id, date"
    ##new_trends = pd.read_sql_query(new_trends_string, engine)

    new_trends = new_trends_all.loc[(new_trends_all['date_request'] > date_lower) & (new_trends_all['date_request'] <= date_upper)]
    
    #closes = closes_all.loc[(closes_all['date_request'] > date_price_lower) & (closes_all['date_request'] <= date_upper)]
    #update to ensure that if the count increased inbetween the request for count and request for data, does not re-check
    #trend_table_rowcount_old = pd.read_sql_query("show table status like 'crypto_trend'", engine)['Rows'].values[0]

    ####################################################################################################################################

    

    if new_trends.empty:
        print(f"Dataframe empty for request: {new_trends_string}")
        check_data = False
        waiting = True
        
    date_request = new_trends[['crypto_id', 'date_request']].groupby(['crypto_id']).mean()
    #date_recent_checked = new_trends.date_request.max()

    try:
        trends = new_trends.pivot(index="date", columns="crypto_id", values="hype") #unbeliiiiieveevevevable !! https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html
    except:
        pd.set_option("display.max_rows", None, "display.max_columns", None)
        print(new_trends)
        

    #check buying and selling conditions
    check_100 = ((trends.iloc[-2] == 100) | (trends.iloc[-1] == 100)) ##check trends with current time
    check_non0 = trends.iloc[0:-2].mean() > 7 # 0:-2
    check_peak = (trends.iloc[0:-10] < buy_threshold).all()
    trend_indicate_buy = np.logical_and.reduce((check_100, check_non0, check_peak), dtype=bool)

    curr_id_list = trends.columns.values
    check_upward_ids = [curr_id_list[i] for i in [i for i, x in enumerate(trend_indicate_buy) if x]]
    

    buy_ids = []
    if check_upward_ids:
        #print(f"Indicate buy ids: {check_upward_ids} between {date_last_checked} : {date_recent_checked}")
        closes = pd.DataFrame()
        for key, crypto_id in enumerate(check_upward_ids):
            date_lower_str = str(date_request.loc[crypto_id]['date_request'] - datetime.timedelta(hours=2))
            date_upper_str = str(date_request.loc[crypto_id]['date_request'])
            #new_close_string = f"SELECT date, close FROM crypto_price WHERE crypto_id = {crypto_id} AND date > '{date_lower_str}' AND date <= '{date_upper_str}' ORDER BY date"
            #close = pd.read_sql_query(new_close_string, engine)
            
            ################ change from store all close to request ################
            close = closes_all.loc[ (closes_all['crypto_id'] == crypto_id) & (closes_all['date'] > date_lower_str) & (closes_all['date'] <= date_upper_str) ]
            #temp_close_string = f"SELECT crypto_id, date, close FROM crypto_price WHERE date >= '{date_lower_str}' AND date <= '{date_upper_str}' AND crypto_id = {crypto_id} ORDER BY date"
            #close = pd.read_sql_query(temp_close_string, engine) 
            ########################################################################

            close.index = pd.to_datetime(close['date'])
            close = close.drop(columns=['date', 'crypto_id'])
            close = close.rename(columns={"close": f"{crypto_id}"})

            if close.empty:
                print(f"Close emtpy for id {crypto_id} at buy at {date_lower_str} - {date_upper_str}")
            else:
                if (closes.empty):
                    closes = close
                else:
                    closes = closes.join(close)
        
        if(closes.empty):
            print("Closes empty")
            continue
        increase = closes.diff(periods=6)
        check_increase = (increase.iloc[-1] >= 0)
        
        

        indicate_buy = check_increase[check_increase].index.values.astype(int)
        #print(f"Indicate buy ids: {indicate_buy} between {date_last_checked} : {date_recent_checked}")
        
        curr_NOT_holding_ids = [full_id_list[i] for i in [i for i, x in enumerate(np.logical_not(holding)) if x]]
        buy_ids = list(set(indicate_buy).intersection(curr_NOT_holding_ids))

        

    check_decrease = np.logical_or((trends.iloc[-1] < sell_threshold), (trends.iloc[-2] < sell_threshold))
    indicate_sell = check_decrease[check_decrease].index.values.astype(int)
    #print(f"Indicate sell ids: {indicate_sell} between {date_last_checked} : {date_recent_checked}")

    curr_holding_ids = [full_id_list[i] for i in [i for i, x in enumerate(holding) if x]]
    sell_ids = list(set(indicate_sell).intersection(curr_holding_ids))


    # Buy control
    if ((sum(holding) < maxholds) and buy_ids):
        for id in buy_ids:
            if (sum(holding) < maxholds):
                key = full_id_list.index(id)
                tradetime = closes.index[-1]
                buyPrice[key] = closes.loc[tradetime,str(id)]
                buyTime[key] = tradetime
                holding[key] = True
                #print(f"Purchased {crypto_table.loc[crypto_table['id'] == id]['ticker'].values[0]} id {id}  at {buyPrice[key]} - {tradetime}")
            else:
                continue
            
    # Sell control
    if (sell_ids):
        for id in sell_ids:
            date_lower_str = str(date_request.loc[id]['date_request'] - datetime.timedelta(hours=2))
            date_upper_str = str(date_request.loc[id]['date_request'])
            #new_close_string = f"SELECT date, close FROM crypto_price WHERE crypto_id = {id} AND date > '{date_lower_str}' AND date <= '{date_upper_str}' ORDER BY date"
            #close = pd.read_sql_query(new_close_string, engine)       
            
            
            ################ change from store all close to request ################
            close = closes_all.loc[ (closes_all['crypto_id'] == id) & (closes_all['date'] > date_lower_str) & (closes_all['date'] <= date_upper_str) ]
            #temp_close_string = f"SELECT crypto_id, date, close FROM crypto_price WHERE date >= '{start_string_close}' AND date <= '{end_string}' AND crypto_id = {crypto_id} ORDER BY date"
            #close = pd.read_sql_query(temp_close_string, engine) 
            ########################################################################

            close.index = pd.to_datetime(close['date'])
            close = close.drop(columns=['date', 'crypto_id'])

            if close.empty:
                print(f"Close emtpy for id at sell {id} at {date_lower_str} - {date_upper_str}")

            # Sell execute function binance using ticker
            key = full_id_list.index(id)
            tradetime = close.index[-1]
            sellPrice[key] = close.loc[tradetime, 'close']
            sellTime[key] = tradetime
            #print(f"Trade completed: {crypto_table.loc[crypto_table['id'] == id]['ticker'].values[0]} id = {id} | open = {buyPrice[key]} at {buyTime[key]} | close = {sellPrice[key]} at {sellTime[key]} | profit = {(sellPrice[key]/buyPrice[key])}%")

            # upload to database
            quantity = 0
            trade = (id, buyTime[key], buyPrice[key], sellTime[key], sellPrice[key], str(sellTime[key] - buyTime[key]),  sellPrice[key]/buyPrice[key], quantity)
            index_names = ['crypto_id', 'date_buy', 'price_buy', 'date_sell', 'price_sell', 'trade_time', 'trade_return', 'quantity']
            result = pd.DataFrame([trade],columns=index_names)
            #print(result)
            tradedata.append([id, buyTime[key], sellTime[key], buyPrice[key], sellPrice[key], (sellPrice[key]/buyPrice[key]) ])
            holding[key] = False

Close emtpy for id 21 at buy at 2022-04-14 06:05:39 - 2022-04-14 08:05:39


In [23]:
tradedata

[[30,
  Timestamp('2022-04-03 14:00:00'),
  Timestamp('2022-04-03 17:00:00'),
  16.5,
  16.46,
  0.9975757575757577],
 [150,
  Timestamp('2022-04-03 15:00:00'),
  Timestamp('2022-04-03 19:00:00'),
  0.009054,
  0.009287,
  1.0257344819969074],
 [107,
  Timestamp('2022-04-03 19:00:00'),
  Timestamp('2022-04-03 20:00:00'),
  0.468,
  0.4666,
  0.997008547008547],
 [51,
  Timestamp('2022-04-03 20:00:00'),
  Timestamp('2022-04-03 21:00:00'),
  9.42,
  9.61,
  1.0201698513800423],
 [257,
  Timestamp('2022-04-03 22:00:00'),
  Timestamp('2022-04-03 23:00:00'),
  4.057,
  4.057,
  1.0],
 [230,
  Timestamp('2022-04-03 22:00:00'),
  Timestamp('2022-04-04 00:00:00'),
  0.30946,
  0.30014,
  0.9698830220383895],
 [8,
  Timestamp('2022-04-04 00:00:00'),
  Timestamp('2022-04-04 02:00:00'),
  1.711,
  1.694,
  0.9900642898889538],
 [154,
  Timestamp('2022-04-04 02:00:00'),
  Timestamp('2022-04-04 03:00:00'),
  525.94,
  528.98,
  1.0057801270106856],
 [112,
  Timestamp('2022-04-04 03:00:00'),
  Times

In [24]:
gains = [(x[0], x[5]) for x in np.array(tradedata)]
sorted(gains, key = lambda x : x[1])

[(177, 0.8729565623540402),
 (102, 0.905429071803853),
 (265, 0.9145869947275923),
 (253, 0.9172510518934082),
 (73, 0.9332905708787685),
 (251, 0.9507163323782235),
 (81, 0.9507865735677631),
 (3, 0.9559082892416225),
 (153, 0.9630808402291534),
 (49, 0.9649256576439191),
 (25, 0.9652239939255884),
 (106, 0.9656188605108055),
 (230, 0.9698830220383895),
 (168, 0.971291866028708),
 (298, 0.972123138713788),
 (153, 0.9743107769423558),
 (252, 0.9826388888888888),
 (4, 0.9827134625458355),
 (254, 0.9845899834892679),
 (277, 0.9848073792729246),
 (251, 0.9855548260013132),
 (268, 0.9855972634800612),
 (256, 0.9857690202517789),
 (170, 0.9858663260962112),
 (66, 0.9859571322985956),
 (159, 0.9871336669049321),
 (112, 0.9887319925830839),
 (3, 0.9891304347826088),
 (8, 0.9897879025923018),
 (220, 0.9899497487437185),
 (8, 0.9900642898889538),
 (74, 0.9901261656609984),
 (36, 0.9910277324632953),
 (106, 0.9915522703273496),
 (161, 0.9939836699613236),
 (257, 0.9940944881889763),
 (106, 0.994

In [2]:
from binance.client import Client
api_key = 'ihvXWbHvvDrYuroIJOS9ANW6X7tv5OQSYibLLloKyIGui0w93Q5yUJOBZclnNL5D'
api_secret = 'uzmKBJ3lmVS4TsTl2AOHVJOYTFfLM5qrg00vAqUyTYbSjiy7BDPlTpaq2Oy7qdo2'
client = Client(api_key, api_secret)
liveprice = client.get_symbol_ticker(symbol='1INCHUSDT')['price']

In [7]:
float(liveprice)*0.99

1.31967

In [7]:
closes

,31,67,144
date,,,
2022-04-14 05:15:00,1.163,0.352,0.05930
2022-04-14 05:30:00,1.187,0.352,0.05921
2022-04-14 05:45:00,1.215,0.352,0.05933
2022-04-14 06:00:00,1.255,0.352,0.05916
2022-04-14 06:15:00,1.260,0.349,0.05945
2022-04-14 06:30:00,1.280,0.348,0.05943
2022-04-14 06:45:00,1.285,0.348,0.05939
2022-04-14 07:00:00,1.280,0.349,0.05930


In [8]:
close

,31
date,
2022-04-14 06:15:00,1.260
2022-04-14 06:30:00,1.280
2022-04-14 06:45:00,1.285
2022-04-14 07:00:00,1.280
2022-04-14 07:15:00,1.309
2022-04-14 07:30:00,1.297
2022-04-14 07:45:00,1.284
2022-04-14 08:00:00,1.272


In [10]:
check_upward_ids

[21, 31, 60, 106, 215, 220, 252, 257, 272]

In [12]:
closes_all.loc[ (closes_all['crypto_id'] == 21)]

,crypto_id,date,close


In [15]:
pd.DataFrame()

""
